In [1]:
import os
import sys
import shutil
import subprocess
import torch

# --- CHECKS ---
if not torch.cuda.is_available():
    raise RuntimeError("❌ NO GPU DETECTED! Please enable 'Accelerator: GPU' in your notebook settings.")

print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")

# --- CLEANUP ---
if os.path.exists('robust-unsupervised'): shutil.rmtree('robust-unsupervised')
if os.path.exists('stylegan3'): shutil.rmtree('stylegan3')

# --- INSTALLATION ---
print("⬇️ Installing pinned dependencies (This takes ~2 mins)...")
# We force specific versions to prevent the NumPy 2.0 crash
subprocess.check_call([
    sys.executable, '-m', 'pip', 'install', 
    'numpy==1.26.4',
    'scipy==1.11.4',
    'ninja', 
    'tyro', 
    'deepface', 
    'lpips',
    'torchvision'
])

print("⬇️ Cloning Repositories...")
subprocess.run(['git', 'clone', 'https://github.com/yohan-pg/robust-unsupervised.git'], stdout=subprocess.DEVNULL)
subprocess.run(['git', 'clone', 'https://github.com/NVlabs/stylegan3.git'], stdout=subprocess.DEVNULL)

print("✅ Environment Ready.")

✅ GPU Detected: Tesla P100-PCIE-16GB
⬇️ Installing pinned dependencies (This takes ~2 mins)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but

⬇️ Cloning Repositories...


Cloning into 'robust-unsupervised'...
Cloning into 'stylegan3'...


✅ Environment Ready.


In [2]:
import sys
import os
import pickle

# Add StyleGAN3 to python path
sys.path.insert(0, os.path.abspath("stylegan3"))
import dnnlib 

print("⬇️ Downloading StyleGAN3 Network...")
network_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhq-1024x1024.pkl'

with dnnlib.util.open_url(network_url) as f:
    net = pickle.load(f)

# Save locally
with open('stylegan3-ffhq.pkl', 'wb') as f:
    pickle.dump(net, f)

print("✅ Model Saved.")

⬇️ Downloading StyleGAN3 Network...
✅ Model Saved.


In [3]:
loss_content = """
import torch
import torch.nn as nn
from lpips import LPIPS
from deepface import DeepFace

def preprocess_for_face_recognition(tensor_image):
    # Scale [-1, 1] to [0, 255]
    image_np = (tensor_image.permute(0, 2, 3, 1) * 127.5 + 127.5).clamp(0, 255).to(torch.uint8).cpu().numpy()
    return image_np

class IDLoss(nn.Module):
    def __init__(self):
        super(IDLoss, self).__init__()
        # DeepFace loads Keras model internally
        self.model = DeepFace.build_model('ArcFace')

    def forward(self, x, y):
        try:
            x_np = preprocess_for_face_recognition(x)
            y_np = preprocess_for_face_recognition(y)
            
            # Get embeddings
            x_emb = DeepFace.represent(img_path=x_np[0], model=self.model, enforce_detection=False)[0]['embedding']
            y_emb = DeepFace.represent(img_path=y_np[0], model=self.model, enforce_detection=False)[0]['embedding']
            
            x_val = torch.tensor(x_emb).unsqueeze(0).to(x.device)
            y_val = torch.tensor(y_emb).unsqueeze(0).to(y.device)
            
            # Cosine Similarity
            return 1 - torch.cosine_similarity(x_val, y_val).mean()
        except:
            return torch.tensor(0.0, device=x.device, requires_grad=True)

class LossFunction(nn.Module):
    def __init__(self, lambda_l1: float = 0.1, lambda_id: float = 0.1):
        super().__init__()
        self.lambda_l1 = lambda_l1
        self.lambda_id = lambda_id
        self.l1 = nn.L1Loss()
        self.lpips = LPIPS(net="vgg").cuda()
        self.id_loss = IDLoss().cuda()

    def forward(self, synth_images, target_images, original_images):
        l1_loss = self.l1(synth_images, target_images)
        lpips_loss = self.lpips(synth_images, target_images).mean()
        id_loss = self.id_loss(synth_images, original_images)
        
        total_loss = lpips_loss + self.lambda_l1 * l1_loss + self.lambda_id * id_loss
        
        log_dict = {
            "total": total_loss.item(),
            "lpips": lpips_loss.item(),
            "l1": l1_loss.item(),
            "id": id_loss.item()
        }
        return total_loss, log_dict
"""

with open('robust-unsupervised/loss_function.py', 'w') as f:
    f.write(loss_content)
print("✅ loss_function.py created.")

✅ loss_function.py created.


In [4]:
script_content = """
import sys
import os
import copy
import torch
import pickle
import numpy as np
from torch.optim import Adam
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import glob
from tqdm import tqdm

# Add SG3 to path
sys.path.insert(0, "../stylegan3")
import dnnlib

# Import our custom loss
from loss_function import LossFunction

def read_image(path, resolution):
    img = Image.open(path).convert('RGB')
    img = img.resize((resolution, resolution), Image.LANCZOS)
    img = TF.to_tensor(img) # [0, 1]
    img = (img * 2) - 1    # [-1, 1]
    return img.unsqueeze(0).cuda()

def save_image(tensor, path):
    img = (tensor.clamp(-1, 1) + 1) / 2
    img = (img * 255).permute(0, 2, 3, 1).detach().cpu().numpy().astype(np.uint8)[0]
    Image.fromarray(img).save(path)

def project_image(G, target, original, steps=500):
    device = target.device
    
    # 1. Initialize Latent (W)
    w_avg = G.mapping.w_avg.clone().detach()
    w = w_avg.unsqueeze(0).repeat(target.shape[0], 1)
    w.requires_grad_(True)
    
    # 2. Initialize Geometric Transform (Novelty: Affine Matrix 2x3)
    # Identity matrix [1, 0, 0; 0, 1, 0]
    theta = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], device=device).unsqueeze(0).repeat(target.shape[0], 1, 1)
    theta.requires_grad_(True)
    
    # Optimizer
    optimizer = Adam([
        {'params': [w], 'lr': 0.1},
        {'params': [theta], 'lr': 0.005}
    ])
    
    loss_fn = LossFunction()
    
    pbar = tqdm(range(steps), leave=False)
    for i in pbar:
        # A. Generate Face
        w_in = w.unsqueeze(1).repeat(1, G.num_ws, 1)
        synth = G.synthesis(w_in, force_fp32=True)
        
        # B. Apply Geometric Alignment (Novelty)
        # We use grid_sample to rotate/shift the generated image to match the target
        grid = F.affine_grid(theta, synth.size(), align_corners=False)
        synth_aligned = F.grid_sample(synth, grid, align_corners=False)
        
        # C. Calculate Loss
        loss, log_dict = loss_fn(synth_aligned, target, original)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f"Loss: {loss.item():.4f} | LPIPS: {log_dict['lpips']:.4f}")
        
    return synth_aligned.detach()

def main():
    # Configuration
    dataset_path = "datasets/samples"
    network_pkl = "../stylegan3-ffhq.pkl"
    outdir = "out_sg3"
    resolution = 1024
    
    print(f"Loading Network: {network_pkl}")
    with open(network_pkl, 'rb') as f:
        G = pickle.load(f)['G_ema'].cuda()
        
    os.makedirs(outdir, exist_ok=True)
    
    # Find images
    files = sorted(glob.glob(f"{dataset_path}/*.png") + glob.glob(f"{dataset_path}/*.jpg"))
    if not files:
        print("No images found! Please check dataset path.")
        return

    print(f"Found {len(files)} images. Starting Restoration...")
    
    for i, path in enumerate(files):
        name = os.path.basename(path)
        print(f"\\nProcessing [{i+1}/{len(files)}]: {name}")
        
        # Load Data
        original = read_image(path, resolution)
        target = original.clone() # In a real scenario, this would be the degraded image
        
        # Run Optimization
        restored = project_image(G, target, original)
        
        # Save Result
        save_path = os.path.join(outdir, f"restored_{name}")
        save_image(restored, save_path)
        print(f"✅ Saved to {save_path}")

if __name__ == "__main__":
    main()
"""

with open('robust-unsupervised/run_stylegan3.py', 'w') as f:
    f.write(script_content)
print("✅ run_stylegan3.py created.")

✅ run_stylegan3.py created.


In [5]:
%cd /kaggle/working/robust-unsupervised

print("🚀 STARTING FULL RESTORATION...")
!python run_stylegan3.py

/kaggle/working/robust-unsupervised
🚀 STARTING FULL RESTORATION...
2025-11-22 19:05:11.824439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763838312.012930      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763838312.069199      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
25-11-22 19:05:26 - Directory /root/.deepface has been created
25-11-22 19:05:26 - Directory /root/.deepface/weights has been created
Loading Network: ../stylegan3-ffhq.pkl
Found 2 images. Starting Restoration...

Processing [1/2]: sample_1.png
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Downloading: "https://download.pytorch.org/models/vgg1